In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from imblearn.over_sampling import SMOTE
import subprocess
import requests
import json
import os

# 1. Create the dataset from the image data
def create_dataset():
    """Create a dataset from the compounds in the image"""
    data = {
        'Drug_ID': ['Drug 1', 'Drug 2', 'Drug 3', 'Drug 4', 'Drug 6'],
        'Drug': [
            'O=[N+]([O-])c1c2c(c3ccc4cccc5ccc3c45)CCCC2',
            'O=c1c2ccccc2c(=O)c2c1ccc1c2[nH]c2c3c(=O)c4cccc...',  # Note: truncated in image
            '[N-]=[N+]=CC(=O)NCC(=O)NN',
            '[N-]=[N+]=C1C=NC(=O)NC1=O',
            'CCCCN(CC(O)C1=CC(=[N+]=[N-])C(=O)C=C1)N=O'
        ],
        'Y': [1, 0, 1, 1, 1]
    }
    return pd.DataFrame(data)

# 2. Set up Ersilia and fetch models
def setup_ersilia(model_id):
    """Set up Ersilia and fetch the specified model"""
    # Initialize Ersilia
    subprocess.run(["ersilia", "init"], check=True)
    
    # Fetch the model
    subprocess.run(["ersilia", "fetch", model_id], check=True)
    
    # Serve the model
    subprocess.run(["ersilia", "serve", model_id], check=True)
    
    return True

# 3. Generate features using Ersilia models
def generate_features(smiles_list, port=3000):
    """Generate molecular features using Ersilia API"""
    features_list = []
    for smile in smiles_list:
        response = requests.post(
            f"http://localhost:{port}/predict",
            json={"input": [smile]}
        )
        features = response.json()["output"][0]
        features_list.append(features)
    
    return np.array(features_list)

# 4. Apply SMOTE to handle class imbalance
def apply_smote(X, y):
    """Apply SMOTE to handle class imbalance"""
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    return X_resampled, y_resampled

# 5. Evaluate model using leave-one-out cross-validation
def evaluate_model(X, y):
    """Evaluate model using leave-one-out cross-validation"""
    # For extremely small datasets, use leave-one-out cross-validation
    loo = LeaveOneOut()
    clf = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
    
    # Use balanced accuracy since classes are imbalanced
    scores = cross_val_score(clf, X, y, cv=loo, scoring='balanced_accuracy')
    
    print(f"Leave-one-out cross-validation scores: {scores}")
    print(f"Mean balanced accuracy: {scores.mean():.4f}")
    
    # Also fit a model on all data (for external validation)
    clf.fit(X, y)
    
    return clf, scores.mean()

# 6. Extract important chemical features
def extract_important_features(model, ersilia_model_id):
    """Extract important features based on the model"""
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        indices = np.argsort(importances)[::-1]
        
        print("Top 10 most important features:")
        for i in range(min(10, len(importances))):
            print(f"Feature {indices[i]}: {importances[indices[i]]:.4f}")
    
    return None

# 7. Main function
def main():
    # Create dataset
    print("Creating dataset from image data...")
    data = create_dataset()
    print(f"Dataset shape: {data.shape}")
    
    # Extract SMILES and targets
    smiles_list = data['Drug'].tolist()
    y = data['Y'].values
    
    # Models to try
    model_options = [
        "eos2ta5",  # Morgan fingerprints (simpler, better for small datasets)
        "eos2r5r",  # Molecular descriptors (interpretable features)
        "eos4rbt"   # Chemical language model embeddings (may overfit with small data)
    ]
    
    results = {}
    for model_id in model_options:
        print(f"\nSetting up Ersilia model: {model_id}")
        setup_ersilia(model_id)
        
        print("Generating features...")
        X = generate_features(smiles_list)
        
        # Check dimensions
        print(f"Feature matrix shape: {X.shape}")
        
        # Skip SMOTE if dimensions are incompatible
        if X.shape[1] >= 2:  # SMOTE requires at least 2 features
            print("Applying SMOTE to handle class imbalance...")
            X_resampled, y_resampled = apply_smote(X, y)
            print(f"Resampled data shape: {X_resampled.shape}")
        else:
            print("Warning: Cannot apply SMOTE with the current feature set")
            X_resampled, y_resampled = X, y
        
        print("Evaluating model...")
        clf, mean_score = evaluate_model(X_resampled, y_resampled)
        
        # Extract important features
        if model_id in ["eos2r5r", "eos2ta5"]:  # Only for interpretable features
            extract_important_features(clf, model_id)
        
        # Store results
        results[model_id] = {
            "mean_score": mean_score,
            "model": clf
        }
        
        # Close the model
        subprocess.run(["ersilia", "close"], check=True)
    
    # Find best model
    if results:
        best_model_id = max(results, key=lambda k: results[k]["mean_score"])
        print(f"\nBest model: {best_model_id} with mean balanced accuracy {results[best_model_id]['mean_score']:.4f}")
    
    # Warning about small dataset
    print("\nWARNING: This model is based on a very small dataset (5 compounds)")
    print("Results should be interpreted with caution and additional validation is strongly recommended")
    print("Consider adding more compounds to the dataset for better predictive performance")

if __name__ == "__main__":
    main()

RuntimeError: Scikit-learn array API support was enabled but scipy's own support is not enabled. Please set the SCIPY_ARRAY_API=1 environment variable before importing sklearn or scipy. More details at: https://docs.scipy.org/doc/scipy/dev/api-dev/array_api.html